In [1]:
import numpy as np
import pandas as pd

In [2]:
trails = pd.read_csv("MaineScrape.csv", encoding = 'mac_roman' )

trails.shape

trails.head()

trails.dtypes

Unnamed: 0          int64
Trail_Name         object
Rank                int64
Difficulty         object
Distance_Miles    float64
Elevation_Feet      int64
Route_Type         object
Description        object
Latitude          float64
Longitude         float64
dtype: object

In [3]:
sum(trails.isnull().values)

#No missing values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
from sklearn import preprocessing

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import OneHotEncoder
from scipy import stats

In [6]:
ntrails = trails.iloc[:,3:7]

ntrailsonehot = trails.loc[:, ['Difficulty', 'Route_Type']]

ntrailsonehot.head()

,Difficulty,Route_Type
0,Hard,Loop
1,Moderate,Out & Back
2,Moderate,Loop
3,Moderate,Loop
4,Hard,Loop


In [7]:
encode = preprocessing.OneHotEncoder()

# 2. Fit the encoder
encode.fit(ntrailsonehot)

# 3. Transform to binary columns
onehotlabels = encode.transform(ntrailsonehot).toarray()
onehotlabels.shape
onehotlabels


array([[0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0., 0.],
       ...,
       [0., 0., 1., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0.]])

In [8]:
newcols = ['Cat1a', 'Cat1b', 'Cat1c', 'Cat2a', 'Cat2b', 'Cat2c']
catvardf = pd.DataFrame(onehotlabels, columns = newcols)

catvardf.shape

(1013, 6)

In [11]:
other_trails = trails.loc[:, ['Distance_Miles', 'Elevation_Feet']]

X = pd.concat((other_trails, catvardf), axis = 1)
X.shape

(1013, 8)

In [12]:
from sklearn.covariance import EllipticEnvelope
outlier_frac = 0.02

ell = EllipticEnvelope(contamination = outlier_frac)
ell.fit(X)

C:\Users\Jordan\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:646: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


EllipticEnvelope(assume_centered=False, contamination=0.02, random_state=None,
         store_precision=True, support_fraction=None)

In [13]:
#Checking for the number of outliers

pred = ell.predict(X)
print(sum(pred == -1) ) # Outlier points are predicted as -1
stats.itemfreq(pred)



21


C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """


array([[ -1,  21],
       [  1, 992]], dtype=int64)

In [14]:
keep = pred == 1

Xkeep = X[keep]

print(Xkeep.shape)
Xkeep.head()

(992, 8)


,Distance_Miles,Elevation_Feet,Cat1a,Cat1b,Cat1c,Cat2a,Cat2b,Cat2c
0,1.4,488,0.0,1.0,0.0,1.0,0.0,0.0
1,4.0,1118,0.0,0.0,1.0,0.0,1.0,0.0
2,3.4,95,0.0,0.0,1.0,1.0,0.0,0.0
3,3.1,597,0.0,0.0,1.0,1.0,0.0,0.0
4,2.1,1049,0.0,1.0,0.0,1.0,0.0,0.0


In [15]:
scaler = StandardScaler()

ready_to_clusterstd = scaler.fit_transform(Xkeep)

C:\Users\Jordan\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Jordan\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [16]:
from sklearn.decomposition import PCA

pca = PCA(0.90)
pca.fit(ready_to_clusterstd)

pca.n_components_

5

In [17]:
pca_components = pca.transform(ready_to_clusterstd)
pca_components.shape

(992, 5)

In [18]:
Xkeep["index"] = Xkeep.index
Xkeep2 = Xkeep.reset_index()
Xkeep2.head()

C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,level_0,Distance_Miles,Elevation_Feet,Cat1a,Cat1b,Cat1c,Cat2a,Cat2b,Cat2c,index
0,0,1.4,488,0.0,1.0,0.0,1.0,0.0,0.0,0
1,1,4.0,1118,0.0,0.0,1.0,0.0,1.0,0.0,1
2,2,3.4,95,0.0,0.0,1.0,1.0,0.0,0.0,2
3,3,3.1,597,0.0,0.0,1.0,1.0,0.0,0.0,3
4,4,2.1,1049,0.0,1.0,0.0,1.0,0.0,0.0,4


In [19]:
cols = []
for i in range (1, 6):
    cols.append("PC" + str(i))

print(cols)

PCdf = pd.DataFrame(pca_components, columns = cols, index  =  Xkeep.index)
PCdf.head()

['PC1', 'PC2', 'PC3', 'PC4', 'PC5']


,PC1,PC2,PC3,PC4,PC5
0,0.421829,1.685960,1.690911,-0.912417,-2.431638
1,1.709415,-0.981107,-1.136659,-0.353078,0.197947
2,-0.217829,1.343543,-1.417123,-0.269028,0.080668
3,0.067842,1.439754,-1.288501,-0.357195,0.046577
4,0.878991,1.851493,1.917381,-0.928493,-2.218600


In [20]:

labels = []
for i in range(5,13):
    kmeans = KMeans(n_clusters= i)
    kmeans = kmeans.fit(PCdf)
    labelPCdf = kmeans.labels_
    labels.append(labelPCdf)
    centroids = kmeans.cluster_centers_
    centroids[:, 0]
    print(stats.itemfreq(labelPCdf))
   


C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and 

[[  0 464]
 [  1 196]
 [  2  85]
 [  3 219]
 [  4  28]]
[[  0  85]
 [  1 285]
 [  2 196]
 [  3 219]
 [  4 179]
 [  5  28]]
[[  0 216]
 [  1 284]
 [  2 180]
 [  3  84]
 [  4 175]
 [  5  28]
 [  6  25]]
[[  0 216]
 [  1 284]
 [  2  43]
 [  3 180]
 [  4 175]
 [  5  41]
 [  6  28]
 [  7  25]]
[[  0 284]
 [  1 110]
 [  2  41]
 [  3 217]
 [  4 175]
 [  5  28]
 [  6  43]
 [  7  16]
 [  8  78]]
[[  0 160]
 [  1 284]
 [  2  15]
 [  3 175]
 [  4 109]
 [  5  41]
 [  6  28]
 [  7  58]
 [  8  79]
 [  9  43]]
[[  0 160]
 [  1 284]
 [  2  18]
 [  3 109]
 [  4 175]
 [  5  28]
 [  6  23]
 [  7  15]
 [  8  43]
 [  9  79]
 [ 10  58]]
[[  0 284]
 [  1 165]
 [  2  79]
 [  3 157]
 [  4  43]
 [  5  27]
 [  6  29]
 [  7  10]
 [  8  53]
 [  9 109]
 [ 10  13]
 [ 11  23]]


C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.


In [21]:
print(labels[1])

[0 2 3 3 0 0 4 3 0 1 3 3 0 2 3 3 1 2 2 0 0 3 3 4 3 2 3 3 3 1 4 3 4 3 0 2 0
 3 3 1 3 2 0 4 1 0 3 2 1 0 0 1 2 2 2 4 1 3 4 2 0 0 2 3 2 1 2 0 0 0 2 0 1 2
 2 3 3 3 3 3 0 2 1 3 0 4 0 3 3 3 1 2 1 2 3 1 2 2 4 0 0 2 3 2 3 3 5 0 2 4 4
 0 4 3 4 0 0 2 2 4 0 3 1 2 3 1 4 1 1 2 0 4 4 3 3 1 2 3 1 0 2 4 1 3 4 2 0 3
 2 2 3 1 0 2 2 1 4 3 3 3 2 3 1 2 2 3 4 3 1 0 3 3 1 2 0 3 4 4 1 4 3 3 3 0 3
 0 0 3 2 3 2 2 3 1 3 2 1 0 2 2 2 3 0 1 4 4 1 1 3 0 4 1 0 2 4 0 4 4 1 1 2 1
 3 3 3 2 2 1 3 3 2 2 2 2 1 4 3 2 4 2 3 3 2 1 2 0 0 1 2 2 2 1 1 3 2 4 4 3 2
 0 1 5 3 1 3 3 3 0 1 3 1 3 2 1 2 4 3 0 3 1 3 3 0 3 1 4 2 3 2 0 2 2 4 2 1 2
 1 0 1 4 1 1 1 3 0 5 1 2 3 3 1 2 5 2 2 2 3 1 1 1 4 4 4 1 2 1 2 1 2 4 4 2 0
 0 3 0 4 1 1 4 3 1 3 4 0 2 3 2 1 3 0 3 1 2 1 1 2 3 2 4 4 1 1 2 1 3 1 2 1 2
 2 4 2 2 3 2 2 1 3 1 4 3 3 3 3 2 2 1 3 2 4 2 4 2 3 2 1 3 2 5 1 1 1 1 4 3 1
 1 3 3 1 4 1 3 3 2 2 3 0 1 3 2 2 2 4 2 4 3 1 0 3 3 5 1 1 4 3 5 4 4 1 3 4 2
 1 4 2 3 1 1 3 0 0 4 5 4 1 2 2 3 3 1 1 0 3 0 2 1 0 2 3 3 3 3 4 4 1 2 3 1 4
 5 2 2 3 1 1 4 2 0 1 1 3 

In [22]:
#Let's go with 6 cluster groups
whole_cluster = labels[1]
whole_cluster.view()

PCdf["Cluster_number"] = whole_cluster
print(PCdf.head())
PCdf["Cluster_number"].value_counts()

        PC1       PC2       PC3       PC4       PC5  Cluster_number
0  0.421829  1.685960  1.690911 -0.912417 -2.431638               0
1  1.709415 -0.981107 -1.136659 -0.353078  0.197947               2
2 -0.217829  1.343543 -1.417123 -0.269028  0.080668               3
3  0.067842  1.439754 -1.288501 -0.357195  0.046577               3
4  0.878991  1.851493  1.917381 -0.928493 -2.218600               0


1    285
3    219
2    196
4    179
0     85
5     28
Name: Cluster_number, dtype: int64

In [23]:
completetrails = pd.concat([trails, PCdf], axis = 1)
completetrails.tail()

,Unnamed: 0,Trail_Name,Rank,Difficulty,Distance_Miles,Elevation_Feet,Route_Type,Description,Latitude,Longitude,PC1,PC2,PC3,PC4,PC5,Cluster_number
1008,1009,Corinna Boardwalk [CLOSED],1009,Easy,0.6,6,Loop,Corinna Boardwalk [CLOSED] is a 0.6 mile moder...,44.92228,-69.25973,-2.105159,0.390970,0.519763,-0.033098,-0.237612,1.0
1009,1010,Bond Mountain [PRIVATE PROPERTY],1010,Moderate,1.7,364,Out & Back,Bond Mountain [PRIVATE PROPERTY] is a 1.7 mile...,43.62735,-70.89608,0.913908,-1.279755,-1.549657,-0.439725,-0.418093,2.0
1010,1011,Porcupine and Notch Trail Loop [CLOSED],1011,Moderate,3.6,275,Loop,Porcupine and Notch Trail Loop [CLOSED] is a 3...,43.21542,-70.68492,-0.074423,1.395277,-1.346425,-0.276145,0.143055,3.0
1011,1012,Shell Pond Loop from Evergreen Valley Road [PR...,1012,Easy,3.7,232,Loop,Shell Pond Loop from Evergreen Valley Road is ...,44.24252,-70.96068,-1.545579,0.615573,0.836192,0.184868,0.531807,1.0
1012,1013,Laughing Lion Trail,1013,Easy,1.7,1148,Out & Back,Laughing Lion Trail is a 1.7 mile moderately t...,44.29391,-70.99949,-0.034112,-1.878868,0.876356,-0.092561,0.010128,4.0


In [27]:
completetrails = completetrails.drop(['Unnamed: 0'], axis = 1)

In [62]:
completetrails.head()

completetrails["State"] = "Maine"
ncompletetrails = completetrails.loc[:,['Trail_Name','Rank','Difficulty','Distance_Miles','Elevation_Feet','Route_Type',
                                     'Description','Cluster_number', 'State','Latitude','Longitude']]


ncompletetrails

,Trail_Name,Rank,Difficulty,Distance_Miles,Elevation_Feet,Route_Type,Description,Cluster_number,State,Latitude,Longitude
0,The Beehive Loop Trail,1,Hard,1.4,488,Loop,The Beehive Loop Trail is a 1.4 mile heavily t...,0.0,Maine,44.33153,-68.18520
1,Cadillac North Ridge Trail,2,Moderate,4.0,1118,Out & Back,Cadillac North Ridge Trail is a 4 mile heavily...,2.0,Maine,44.37854,-68.22954
2,Jordan Pond Full Loop Trail,3,Moderate,3.4,95,Loop,Jordan Pond Full Loop Trail is a 3.4 mile heav...,3.0,Maine,44.32200,-68.25437
3,Ocean Path and Gorham Mountain Loop Trail,4,Moderate,3.1,597,Loop,Ocean Path and Gorham Mountain Loop Trail is ...,3.0,Maine,44.32998,-68.18404
4,"Precipice, Orange and Black and Champlain Nort...",5,Hard,2.1,1049,Loop,"Precipice, Orange and Black and Champlain Nort...",0.0,Maine,44.34948,-68.18776
5,Cadillac Mountain South Ridge Trail Loop,6,Hard,7.5,2247,Loop,Cadillac Mountain South Ridge Trail Loop is a ...,0.0,Maine,44.31306,-68.21502
6,Ocean Path Trail: Thunder Hole and Monument Cove,7,Easy,4.5,374,Out & Back,Ocean Path Trail: Thunder Hole and Monument Co...,4.0,Maine,44.32985,-68.18432
7,Great Head Trail (Short Option),8,Moderate,1.6,301,Loop,Great Head Trail (Short Option) is a 1.6 mile ...,3.0,Maine,44.33001,-68.18399
8,Tumbledown Mountain Trail,9,Hard,5.3,1952,Loop,Tumbledown Mountain Trail is a 5.3 mile heavil...,0.0,Maine,44.73154,-70.55881
9,Cadillac Summit Loop Trail,10,Easy,0.3,45,Loop,Cadillac Summit Loop Trail is a 0.3 mile heavi...,1.0,Maine,44.35280,-68.22459


In [60]:
nhtrails = pd.read_csv("FinalClusterTrail.csv")

nhtrails['State'] = "New Hampshire"


nnhtrails = nhtrails.loc[:,['Trail_Name','Rank','Difficulty','Distance_Miles','Elevation_Feet','Route_Type',
                                     'Description','Cluster_number', 'State','Latitude','Longitude']]
nnhtrails

,Trail_Name,Rank,Difficulty,Distance_Miles,Elevation_Feet,Route_Type,Description,Cluster_number,State,Latitude,Longitude
0,Mount Lafayette and Franconia Ridge Trail Loop,1,Hard,8.6,3822,Loop,Mount Lafayette and Franconia Ridge Trail Loop...,4.0,New Hampshire,44.14161,-71.68188
1,Welch and Dickey Loop Trail,2,Hard,4.0,1751,Loop,Welch and Dickey Loop Trail is a 4 mile heavil...,4.0,New Hampshire,43.90465,-71.58856
2,Mount Willard Trail,3,Moderate,3.2,905,Out & Back,Mount Willard Trail is a 3.2 mile heavily traf...,3.0,New Hampshire,44.21817,-71.41145
3,Mount Monadnock via White Dot and White Cross ...,4,Hard,3.7,1768,Loop,Mount Monadnock via White Dot and White Cross ...,4.0,New Hampshire,42.84532,-72.08741
4,Mount Major and Brook Trail Loop,5,Moderate,3.7,1148,Loop,Mount Major and Brook Trail Loop is a 3.7 mile...,0.0,New Hampshire,43.51921,-71.27390
5,Tuckerman Ravine Trail to Mount Washington,6,Hard,7.9,4169,Point to Point,Tuckerman Ravine Trail to Mount Washington is ...,2.0,New Hampshire,44.25774,-71.25328
6,Lonesome Lake Trail,7,Moderate,3.1,1026,Loop,Lonesome Lake Trail is a 3.1 mile heavily traf...,0.0,New Hampshire,44.14195,-71.68414
7,Mount Moosilauke and South Peak Loop,8,Hard,7.2,2506,Loop,Mount Moosilauke and South Peak Loop is a 7.2 ...,4.0,New Hampshire,43.99356,-71.81616
8,Champney Falls Trail,9,Moderate,3.1,682,Out & Back,Champney Falls Trail is a 3.1 mile heavily tra...,3.0,New Hampshire,43.98994,-71.29891
9,Arethusa Falls and Frankenstein Cliff Trail,10,Hard,4.2,1538,Loop,Arethusa Falls and Frankenstein Cliff Trail is...,4.0,New Hampshire,44.14809,-71.36629


In [61]:
alltrails = pd.concat([ncompletetrails, nnhtrails])

print(alltrails.shape)
alltrails

(2209, 11)


,Trail_Name,Rank,Difficulty,Distance_Miles,Elevation_Feet,Route_Type,Description,Cluster_number,State,Latitude,Longitude
0,The Beehive Loop Trail,1,Hard,1.4,488,Loop,The Beehive Loop Trail is a 1.4 mile heavily t...,0.0,Maine,44.33153,-68.18520
1,Cadillac North Ridge Trail,2,Moderate,4.0,1118,Out & Back,Cadillac North Ridge Trail is a 4 mile heavily...,2.0,Maine,44.37854,-68.22954
2,Jordan Pond Full Loop Trail,3,Moderate,3.4,95,Loop,Jordan Pond Full Loop Trail is a 3.4 mile heav...,3.0,Maine,44.32200,-68.25437
3,Ocean Path and Gorham Mountain Loop Trail,4,Moderate,3.1,597,Loop,Ocean Path and Gorham Mountain Loop Trail is ...,3.0,Maine,44.32998,-68.18404
4,"Precipice, Orange and Black and Champlain Nort...",5,Hard,2.1,1049,Loop,"Precipice, Orange and Black and Champlain Nort...",0.0,Maine,44.34948,-68.18776
5,Cadillac Mountain South Ridge Trail Loop,6,Hard,7.5,2247,Loop,Cadillac Mountain South Ridge Trail Loop is a ...,0.0,Maine,44.31306,-68.21502
6,Ocean Path Trail: Thunder Hole and Monument Cove,7,Easy,4.5,374,Out & Back,Ocean Path Trail: Thunder Hole and Monument Co...,4.0,Maine,44.32985,-68.18432
7,Great Head Trail (Short Option),8,Moderate,1.6,301,Loop,Great Head Trail (Short Option) is a 1.6 mile ...,3.0,Maine,44.33001,-68.18399
8,Tumbledown Mountain Trail,9,Hard,5.3,1952,Loop,Tumbledown Mountain Trail is a 5.3 mile heavil...,0.0,Maine,44.73154,-70.55881
9,Cadillac Summit Loop Trail,10,Easy,0.3,45,Loop,Cadillac Summit Loop Trail is a 0.3 mile heavi...,1.0,Maine,44.35280,-68.22459


In [73]:
alltrails.to_csv("NHMaineTrails.csv")

In [77]:
alltrails = pd.read_csv("NHMaineTrails.csv")

alltrails.shape

(2209, 11)

In [78]:
nalltrails = alltrails.loc[:,['Rank','Difficulty','Distance_Miles','Elevation_Feet','Route_Type']]
print(nalltrails.shape)
nalltrails

(2209, 5)


,Rank,Difficulty,Distance_Miles,Elevation_Feet,Route_Type
0,1,Hard,1.4,488,Loop
1,2,Moderate,4.0,1118,Out & Back
2,3,Moderate,3.4,95,Loop
3,4,Moderate,3.1,597,Loop
4,5,Hard,2.1,1049,Loop
5,6,Hard,7.5,2247,Loop
6,7,Easy,4.5,374,Out & Back
7,8,Moderate,1.6,301,Loop
8,9,Hard,5.3,1952,Loop
9,10,Easy,0.3,45,Loop


In [79]:
nalltrailsonehot = nalltrails.loc[:, ['Difficulty', 'Route_Type']]
nalltrailsonehot

,Difficulty,Route_Type
0,Hard,Loop
1,Moderate,Out & Back
2,Moderate,Loop
3,Moderate,Loop
4,Hard,Loop
5,Hard,Loop
6,Easy,Out & Back
7,Moderate,Loop
8,Hard,Loop
9,Easy,Loop


In [80]:
encode = preprocessing.OneHotEncoder()

# 2. Fit the encoder
encode.fit(nalltrailsonehot)

# 3. Transform to binary columns
onehotlabels = encode.transform(nalltrailsonehot).toarray()
onehotlabels.shape
onehotlabels


array([[0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.]])

In [81]:
newcols = ['Cat1a', 'Cat1b', 'Cat1c', 'Cat2a', 'Cat2b', 'Cat2c']
catvardf = pd.DataFrame(onehotlabels, columns = newcols)

catvardf

,Cat1a,Cat1b,Cat1c,Cat2a,Cat2b,Cat2c
0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,1.0,1.0,0.0,0.0
8,0.0,1.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,1.0,0.0,0.0


In [96]:
other_trails = nalltrails.loc[:, ['Distance_Miles', 'Elevation_Feet']]

X = pd.concat((other_trails, catvardf), axis = 1)
X

,Distance_Miles,Elevation_Feet,Cat1a,Cat1b,Cat1c,Cat2a,Cat2b,Cat2c
0,1.4,488,0.0,1.0,0.0,1.0,0.0,0.0
1,4.0,1118,0.0,0.0,1.0,0.0,1.0,0.0
2,3.4,95,0.0,0.0,1.0,1.0,0.0,0.0
3,3.1,597,0.0,0.0,1.0,1.0,0.0,0.0
4,2.1,1049,0.0,1.0,0.0,1.0,0.0,0.0
5,7.5,2247,0.0,1.0,0.0,1.0,0.0,0.0
6,4.5,374,1.0,0.0,0.0,0.0,1.0,0.0
7,1.6,301,0.0,0.0,1.0,1.0,0.0,0.0
8,5.3,1952,0.0,1.0,0.0,1.0,0.0,0.0
9,0.3,45,1.0,0.0,0.0,1.0,0.0,0.0


In [97]:
from sklearn.covariance import EllipticEnvelope
outlier_frac = 0.02

ell = EllipticEnvelope(contamination = outlier_frac)
ell.fit(X)

C:\Users\Jordan\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:646: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


EllipticEnvelope(assume_centered=False, contamination=0.02, random_state=None,
         store_precision=True, support_fraction=None)

In [98]:
#Checking for the number of outliers

pred = ell.predict(X)
print(sum(pred == -1) ) # Outlier points are predicted as -1
stats.itemfreq(pred)



45


C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """


array([[  -1,   45],
       [   1, 2164]], dtype=int64)

In [99]:
keep = pred == 1

Xkeep = X[keep]

print(Xkeep.shape)
Xkeep.head()

(2164, 8)


,Distance_Miles,Elevation_Feet,Cat1a,Cat1b,Cat1c,Cat2a,Cat2b,Cat2c
0,1.4,488,0.0,1.0,0.0,1.0,0.0,0.0
1,4.0,1118,0.0,0.0,1.0,0.0,1.0,0.0
2,3.4,95,0.0,0.0,1.0,1.0,0.0,0.0
3,3.1,597,0.0,0.0,1.0,1.0,0.0,0.0
4,2.1,1049,0.0,1.0,0.0,1.0,0.0,0.0


In [100]:
scaler = StandardScaler()

ready_to_clusterstd = scaler.fit_transform(Xkeep)

C:\Users\Jordan\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Jordan\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [101]:
from sklearn.decomposition import PCA

pca = PCA(0.90)
pca.fit(ready_to_clusterstd)

pca.n_components_

4

In [102]:
pca_components = pca.transform(ready_to_clusterstd)
pca_components.shape

(2164, 4)

In [103]:
Xkeep["index"] = Xkeep.index
Xkeep2 = Xkeep.reset_index()
Xkeep2.head()

C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,level_0,Distance_Miles,Elevation_Feet,Cat1a,Cat1b,Cat1c,Cat2a,Cat2b,Cat2c,index
0,0,1.4,488,0.0,1.0,0.0,1.0,0.0,0.0,0
1,1,4.0,1118,0.0,0.0,1.0,0.0,1.0,0.0,1
2,2,3.4,95,0.0,0.0,1.0,1.0,0.0,0.0,2
3,3,3.1,597,0.0,0.0,1.0,1.0,0.0,0.0,3
4,4,2.1,1049,0.0,1.0,0.0,1.0,0.0,0.0,4


In [104]:
cols = []
for i in range (1, 5):
    cols.append("PC" + str(i))

print(cols)

PCdf = pd.DataFrame(pca_components, columns = cols, index  =  Xkeep.index)
PCdf.head()

['PC1', 'PC2', 'PC3', 'PC4']


,PC1,PC2,PC3,PC4
0,0.136529,1.629438,-0.068469,-0.981418
1,0.550061,-1.915989,0.782815,0.034991
2,-0.901937,0.338701,1.881356,-0.012276
3,-0.734393,0.392570,1.883690,-0.068989
4,0.445905,1.739028,-0.063931,-1.021127


In [105]:

labels = []
for i in range(5,13):
    kmeans = KMeans(n_clusters= i)
    kmeans = kmeans.fit(PCdf)
    labelPCdf = kmeans.labels_
    labels.append(labelPCdf)
    centroids = kmeans.cluster_centers_
    centroids[:, 0]
    print(stats.itemfreq(labelPCdf))
   


C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and 

[[  0 750]
 [  1 448]
 [  2  77]
 [  3 509]
 [  4 380]]
[[  0 319]
 [  1 509]
 [  2 435]
 [  3  77]
 [  4 376]
 [  5 448]]
[[  0 319]
 [  1 448]
 [  2 163]
 [  3 508]
 [  4 433]
 [  5 216]
 [  6  77]]
[[  0 448]
 [  1 216]
 [  2 508]
 [  3 433]
 [  4 319]
 [  5  33]
 [  6  44]
 [  7 163]]
[[  0 102]
 [  1 508]
 [  2 209]
 [  3 448]
 [  4 319]
 [  5  44]
 [  6 163]
 [  7  33]
 [  8 338]]
[[  0 448]
 [  1 147]
 [  2 508]
 [  3  33]
 [  4 319]
 [  5 102]
 [  6 163]
 [  7  44]
 [  8  62]
 [  9 338]]
[[  0 338]
 [  1 508]
 [  2 147]
 [  3 448]
 [  4 319]
 [  5  33]
 [  6  86]
 [  7  44]
 [  8  62]
 [  9 102]
 [ 10  77]]
[[  0 338]
 [  1  78]
 [  2 508]
 [  3 319]
 [  4 448]
 [  5  62]
 [  6  33]
 [  7  24]
 [  8  85]
 [  9 102]
 [ 10 147]
 [ 11  20]]


C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Jordan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  # Remove the CWD from sys.path while we load stuff.


In [106]:
#Let's go with 6 cluster groups
whole_cluster = labels[1]
whole_cluster.view()

PCdf["Cluster_number_whole"] = whole_cluster
print(PCdf.head())
PCdf["Cluster_number_whole"].value_counts()

        PC1       PC2       PC3       PC4  Cluster_number_whole
0  0.136529  1.629438 -0.068469 -0.981418                     1
1  0.550061 -1.915989  0.782815  0.034991                     2
2 -0.901937  0.338701  1.881356 -0.012276                     5
3 -0.734393  0.392570  1.883690 -0.068989                     5
4  0.445905  1.739028 -0.063931 -1.021127                     4


1    509
5    448
2    435
4    376
0    319
3     77
Name: Cluster_number_whole, dtype: int64

In [110]:
completetrails = pd.concat([alltrails, PCdf], axis = 1)
completetrails.head()
completetrails.shape

(2209, 16)

In [111]:
completetrails.to_csv("NHMaineCompleteClusteredTrails.csv")